In [3]:
import numpy as np
import math
from Crypto.Util.number import getPrime, inverse, bytes_to_long, long_to_bytes, GCD
import random

# Prerequisites: 

1. RSA 

2. RSA Prerequisites

In [4]:
def create_key_rsa(p, q, e):
    assert GCD(e, (p-1)*(q-1)) == 1
    N = p*q
    return N, e
def encrypt_rsa(m, e, N):
    assert m < N
    c = pow(m, e, N)
    return c
def decrypt_rsa(c, p, q, e, N):
    d = inverse(e, (p-1)*(q-1))
    assert (d * e) % ((p-1)*(q-1)) == 1
    m = pow(c, d, N)
    return m

# $N$ is prime

If N is prime => $\varphi(N) = p-1 \Rightarrow de \equiv 1 \ \bmod \ N-1$

In [5]:
m = bytes_to_long(b'secret')
N = getPrime(1024)
e=65537

In [6]:
c = encrypt_rsa(m, e, N)
print(c)

113595020956362025967702690785507787104609915846283281983495084918991993652665132298948338220454799260319934908771253892734008530993345219868929130126527976702388584607766096693700128436653319865990618419025169755195153556717670189122217486760266907678050675517893440208299707580398786489677325880671984665640


In [7]:
phi = N-1
d = inverse(e, phi)
m_decr = pow(c, d, N)

In [8]:
long_to_bytes(m_decr)

b'secret'

# $N = p^2$

https://en.wikipedia.org/wiki/Euler%27s_totient_function#Value_for_a_prime_power_argument

If $N=p^2$ is prime $\Rightarrow \phi(N) = p \cdot (p-1)$

In [9]:
def isqrt(n):
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

In [10]:
m = bytes_to_long(b'secret')
p = getPrime(512)
N = pow(p, 2)
e=65537

In [11]:
c = encrypt_rsa(m, e, N)
print(c)

26618294004460270697140137636718238354655341120624517318023468816935911695310711773457590282453629675234951871763151713423679349421077571087622047732248791395683409055002063900127590123806673186322126878036258963541114865663994373830716703114391796845413963015455569977117274553717904064728502239857825963182


In [12]:
p_decr = isqrt(N)
phi = p*(p-1)
d = inverse(e, phi)
m_decr = pow(c, d, N)
print(long_to_bytes(m_decr))

b'secret'


# $e = 1$

Then $c \equiv m^e \equiv m^1 \equiv m \ \bmod \ N$

# Using small N

You can factor N => factordb library


In [57]:
from factordb.factordb import FactorDB

In [64]:
p = getPrime(100)
q = getPrime(100)
N = p*q
print('p= ', p)
print('q= ', q)

p=  762519624440665876556479833467
q=  1106090959122647201209128911207


In [76]:
N

843416062747416855803616155832785095979008569880606389964669

In [74]:
f = FactorDB(N)
f.connect()

<Response [200]>

In [75]:
print(f.get_factor_list())

[762519624440665876556479833467, 1106090959122647201209128911207]


# Small e, very big N

In this scenario we might not reach the modulus => just take the e-th root

In [13]:
def iroot(k, n):
    u, s = n, n+1
    while u < s:
        s = u
        t = (k-1) * s + n // pow(s, k-1)
        u = t // k
    return s

In [14]:
e = 3
N = 17258212916191948536348548470938004244269544560039009244721959293554822498047075403658429865201816363311805874117705688359853941515579440852166618074161313773416434156467811969628473425365608002907061241714688204565170146117869742910273064909154666642642308154422770994836108669814632309362483307560217924183202838588431342622551598499747369771295105890359290073146330677383341121242366368309126850094371525078749496850520075015636716490087482193603562501577348571256210991732071282478547626856068209192987351212490642903450263288650415552403935705444809043563866466823492258216747445926536608548665086042098252335883
m = bytes_to_long(b'secret')
c = encrypt_rsa(m, e, N)

In [15]:
m_decr =iroot(3, c)
print(long_to_bytes(m_decr))

b'secret'


# Multiple N's have a common prime 

- Here we have $N_1, N_2 ...$ moduli with $C_1, C_2, ... $ ciphertexts encrypted with $e_1, e_2, ...$ corresponding public exponents
- If we find $\gcd(N_i, N_j) \neq 1$ for some $i\neq j$ then we found $p = \gcd(N_i, N_j) => q_i = \dfrac {N_i}  p; \quad  q_j = \dfrac {N_j} p => (d_i, d_j) => (m_i, m_j)$

In [16]:
p = getPrime(512)
q1 = getPrime(512)
q2 = getPrime(512)
N1 = p*q1
N2 = p*q2
e1, e2 = 17,65537
m1, m2 = bytes_to_long(b'secret'), bytes_to_long(b'verysecret')
c1, c2 = pow(m1, e1, N1), pow(m2, e2, N2)

In [17]:
p_decr = GCD(N1, N2)
q1_decr = N1 // p_decr
q2_decr = N2 // p_decr
d1 = inverse(e1, (p-1)*(q1_decr-1))
d2 = inverse(e2, (p-1)*(q2_decr-1))
m1_decr = pow(c1, d1, N1)
m2_decr = pow(c2, d2, N2)

In [18]:
long_to_bytes(m1_decr), long_to_bytes(m2_decr)

(b'secret', b'verysecret')

# Find $N$ from $(m, c)$ pairs

- https://crypto.stackexchange.com/questions/30289/is-it-possible-to-recover-an-rsa-modulus-from-its-signatures/30301#30301
- https://crypto.stackexchange.com/questions/43583/deduce-modulus-n-from-public-exponent-and-encrypted-data

## Case 1: $e$ is small (and known)
Get 2 pairs $(m_1, c_1), (m_2, c_2)$.  
We know $c_1 = m_1^e \bmod n \iff c_1 - m_1^e \equiv 0 \bmod n \iff c_1 - m_1^e = k_1 \cdot n$ for some $k_1 \in \mathbb Z$  
Similarly $c_2 - m_2^e = k_2 \cdot n$

Therefore we can compute $g = \gcd(c_1 - m_1^e, c_2 - m_2^e) = \gcd(k_1\cdot n, k_2\cdot n) = \gcd(k_1, k_2) \cdot n$  
If $\gcd(k_1, k_2) \neq 1$ we can search through the divisors of $g$ for $n$

In [58]:
# Generate parameters
e = 3 # for time reasons
p = getPrime(512)
q = getPrime(512)
n = p * q

# Pick messages and send to oracle to encrypt
m1 = random.randint(0, n)
m2 = random.randint(0, n)
c1 = pow(m1, e, n)
c2 = pow(m2, e, n)

#n_ = GCD(pow(m1, e) - c1, pow(m2, e) - c2)
n_ = GCD(c1 - pow(m1, e), c2 - pow(m2, e))
n_ == n

False

## Case 2: $e$ is unknown

Pick $m_1, m_2$ and compute   
$m_1' = m_1^2 \\ m_2' = m_2^2$

Encrypt and get $c_1, c_1', c_2, c_2'$

$c_1' \equiv m_1'^e \equiv (m_1^2)^e \equiv (m_1^e)^2 \equiv c_1^2 \bmod n \iff c_1' - c_1^2 \equiv 0 \bmod n \Rightarrow c_1' - c_1^2 = k_1 \cdot n$. Similarly  
$c_2' \equiv c_2^2 \bmod n \Rightarrow c_2' - c_2^2 = k_2 \cdot n$

Therefore we can compute $g = \gcd(c_1' - c_1^2, c_2' - c_2^2) =  \gcd(k_1\cdot n, k_2\cdot n) = \gcd(k_1, k_2) \cdot n$

**Remark:** $c_1' \equiv c_1 ^ 2 \bmod n$ is true but $c_1' = c_1 ^ 2$ is not necesarly true

In [75]:
# Generate parameters
e = 3 # for time reasons
p = getPrime(512)
q = getPrime(512)
n = p * q
print(f'{n = }')
# Pick messages and send to oracle to encrypt
m1 = random.randint(0, n)
m1_ = pow(m1, 2)
m2 = random.randint(0, n)
m2_ = pow(m2, 2)

c1 = pow(m1, e, n)
c1_ = pow(m1_, e, n)
c2 = pow(m2, e, n)
c2_ = pow(m2_, e, n)

# Check our remark
print(f'{c1_ % n == pow(c1, 2, n) = }')
print(f'{c1_ != pow(c1, 2) = } ')


#n_ = GCD(pow(m1, e) - c1, pow(m2, e) - c2)
n_ = GCD(c1_ - pow(c1, 2), c2_ - pow(c2, 2))

# Search small factors if not found
if n_ == n:
    print(f'found {n_ = }')
else:
    for i in range(1, 1000):
        if n_ % i == 0:
            n_ //= i
            if n_ == n:
                print(f'found {n_}')
                break
    

n = 105739742421860841378444126270866531355942007064414792597967440990847031405597804518490107902682444473278375203388226764993567926157825944454621165783518838232835867601771731448588658723495215473663310736510435918878725507291338888131176038727668592712783433913192481748104187454164879453648907820993174348757
c1_ % n == pow(c1, 2, n) = True
c1_ != pow(c1, 2) = True 
found n_ = 105739742421860841378444126270866531355942007064414792597967440990847031405597804518490107902682444473278375203388226764993567926157825944454621165783518838232835867601771731448588658723495215473663310736510435918878725507291338888131176038727668592712783433913192481748104187454164879453648907820993174348757
